# Simple Dialogue State Machine

We will be building a state machine that can hold a conversation like:

```
(System) Enter an animal:
(User) cat
(System) cat is a mammal, enter another animal:
(User) dove
(System) dove is a bird, enter another animal:
(User) kangaroo
(System) i dont know that one, enter another animal:
...
```

## Step 1: Define the States

Define the states that your conversation will use.

You can modify this class as you build your conversation state machine.

Make sure that all states that you use in transitions later are present in this class!

In [1]:
from emora_stdm import KnowledgeBase, DialogueFlow
from enum import Enum

class State(Enum):
    START = 0
    PROMPT = 1
    MAMMAL = 2
    BIRD = 3
    ERR = 4

[nltk_data] Downloading package wordnet to /Users/Sarah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Step 2: Initialize DialogueFlow Object

Initialize your state machine as a DialogueFlow object.

**First argument:**
* the state that your conversation starts in

**Second argument:**
* the speaker who will talk first
   * If SYSTEM: an utterance will be produced on the first turn before waiting for input of user utterance
   * If USER: the first turn will wait for input of user utterance


In [2]:
df = DialogueFlow(State.START, initial_speaker=DialogueFlow.Speaker.SYSTEM)

## Step 3: Define Transitions Between States

Add transitions between states defining what utterances will move from one state to another

**First argument:**
* source state - the state that your conversation is in

**Second argument:**
* target state - the state that the conversation moves to if the NatEx expression matches

**Third argument:**
* Natex - expression defining what types of utterances will match the transition
    * If add_system_transition: Natex defines what is produced 
    * If add_user_transition: Natex defines what the user utterance must be like



### First Turn - START to PROMPT

In [3]:
df.add_system_transition(State.START, State.PROMPT, 'Enter an animal":"')

<img src="res/startprompt.png" />

### Second Turn - PROMPT to MAMMAL

In [4]:
df.add_user_transition(State.PROMPT, State.MAMMAL, "$animal={cat,dog}")

<img src="res/promptmammal.png" />

### Second Turn - PROMPT to BIRD

In [5]:
df.add_user_transition(State.PROMPT, State.BIRD, "$animal={parrot,dove,crow}")

<img src="res/promptbird.png" />

### Looping Turn - MAMMAL to PROMPT

In [6]:
df.add_system_transition(State.MAMMAL, State.PROMPT, '[! $animal is a mammal"," enter another animal":"]')

<img src="res/mammalprompt.png" />

### Looping Turn - BIRD to PROMPT

In [7]:
df.add_system_transition(State.BIRD, State.PROMPT, '[! $animal is a bird"," enter another animal":"]')

<img src="res/birdprompt.png" />

## Special Case: Error Transitions From States

Set up how the state machine will behave if there are no transitions that match the user utterance

**First argument:**
* source state - the state that your conversation is waiting for user utterance in

**Second argument:** 
* target state - the state that the conversation moves to if no transitions from the source state match the user utterance

Every state that has user transitions from it must specify an error successor in this way!


### Error Transition: PROMPT to ERR

In [8]:
df.set_error_successor(State.PROMPT, State.ERR)

<img src="res/prompterror.png" />

### Looping Turn - ERR to PROMPT

In [9]:
df.add_system_transition(State.ERR, State.PROMPT, '[! i dont know that one"," enter another animal":"]')

<img src="res/errorprompt.png" />

## Step 5: Run the DialogueFlow and Talk

When running a DialogueFlow, there is no ending point to the conversation.

To stop the conversation, you must stop the script from running (which will show a KeyboardInterrupt Error and this is expected).
* In iPython/Jupyter Notebook, do this by clicking the stop (square) button

In [10]:
df.run(debugging=False)

S: Enter an animal :
U: cat
S: cat is a mammal , enter another animal :
U: dog
S: dog is a mammal , enter another animal :
U: parrot
S: parrot is a bird , enter another animal :
U: dove
S: dove is a bird , enter another animal :
U: crow
S: crow is a bird , enter another animal :
U: dude
S: i dont know that one , enter another animal :
U: anything
S: i dont know that one , enter another animal :


KeyboardInterrupt: 

## Put it all Together

In [ ]:
from emora_stdm import KnowledgeBase, DialogueFlow
from enum import Enum

# Define the states that your conversation will use
#
# All states that you use in transitions later need to be in this class
#
class State(Enum):
    START = 0
    PROMPT = 1
    MAMMAL = 2
    BIRD = 3
    ERR = 4

# Initialize your state machine like this
#
# first argument: the state that your conversation starts in
# second argument: the speaker who will take the first transition
#    If SYSTEM: an utterance will be produced on the first turn before waiting for input of user utterance
#    If USER: the first turn will wait for input of user utterance
#
df = DialogueFlow(State.START, initial_speaker=DialogueFlow.Speaker.SYSTEM)

# Add transitions between states defining what utterances will move from one state to another
#
# first argument: source state - the state that your conversation is in
# second argument: target state - the state that the conversation moves to if the NatEx expression matches
# third argument: Natex - expression defining what types of utterances will match the transition
#    If add_system_transition: Natex defines what is produced 
#    If add_user_transition: Natex defines what the user utterance must be like
#
df.add_system_transition(State.START, State.PROMPT, 'Enter an animal":"')
df.add_user_transition(State.PROMPT, State.MAMMAL, "$animal={cat,dog}")
df.add_user_transition(State.PROMPT, State.BIRD, "$animal={parrot,dove,crow}")
df.add_system_transition(State.MAMMAL, State.PROMPT, '[! $animal is a mammal"," enter another animal":"]')
df.add_system_transition(State.BIRD, State.PROMPT, '[! $animal is a bird"," enter another animal":"]')
df.add_system_transition(State.ERR, State.PROMPT, '[! i dont know that one"," enter another animal":"]')


# Set up how the state machine will behave if there are no transitions that match the user utterance
#
# first argument: source state - the state that your conversation is waiting for user utterance in
# second argument: target state - the state that the conversation moves to if no transitions from the source state match the user utterance
#
# Every state that has user transitions from it must specify an error successor in this way!
#
df.set_error_successor(State.PROMPT, State.ERR)


df.run(debugging=False)